<a href="https://colab.research.google.com/github/syX113/hslu-nlp/blob/train-colab/stage3/03-train-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLT Project - Stage III





- **Author:**             Arian Contessotto, Tim Giger, Levin Reichmuth
- **Submission Date:**    1 June 2023

## 1. Prerequisites and Load

In [ ]:
# Clone repo with dataset
!git clone https://github.com/syX113/hslu-nlp

Cloning into 'hslu-nlp'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 212 (delta 18), reused 28 (delta 8), pack-reused 122
Receiving objects: 100% (212/212), 40.85 MiB | 7.05 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Updating files: 100% (12/12), done.


In [ ]:
# Check if files are loaded
!ls hslu-nlp/stage2/annotated/

full_llm_annotated.csv	gold_standard.csv


If necessary, install the required packages.

In [ ]:
# Required package installation
!transformers==4.28.0
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 1.1 Import Packages and Make Downloads

In [1]:
# Imports
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score,  mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.tensorboard import SummaryWriter
warnings.filterwarnings("ignore")

2023-05-26 14:57:52.372680: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-26 14:57:52.525578: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 14:57:53.159311: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### 1.2 Load Annotated Data

The final dataframe from stage one is loaded. These data are the basis for stage two.

In [52]:
# Define file name
esg_file = '../stage2/annotated/full_llm_annotated.csv' # Local filepath
#esg_file = 'hslu-nlp/stage2/annotated/full_llm_annotated.csv' # Filepath on Colab

# Define function to load and merge data
def load_data(file):

    # Load the data
    df = pd.read_csv(file, delimiter = '|')

    # Apply eval function
    df['esg_topics'] = df['esg_topics'].apply(eval)
    df['sentence_tokens'] = df['sentence_tokens'].apply(eval)
    df['sentiment_llm_continuous'] = df['sentiment_llm_continuous'].apply(eval)
    df['sentiment_llm_categorial'] = df['sentiment_llm_categorial'].apply(eval)

    return df

df = load_data(esg_file)

# Print shape and diyplay header
print(df.shape)
df.head()

(11071, 17)


,company,datatype,title,date,domain,esg_topics,internal,symbol,sentence_tokens,market_cap_in_usd_b,sector,industry,year_month,year,month,sentiment_llm_continuous,sentiment_llm_categorial
0,Beiersdorf,sustainability_report,BeiersdorfAG Sustainability Report 2021,2021-03-31,NaN,"[CleanWater, GHGEmission, ProductLiability, Va...",1,BEI,[brands strategy sustainability agenda care be...,25.99,Consumer Staples,Household & Personal Products,2021-03,2021,3,"[0.4510161280632019, 0.6138720512390137, 0.226...","[0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 1.0, ..."
1,Deutsche Telekom,sustainability_report,DeutscheTelekomAG Sustainability Report 2021,2021-03-31,NaN,"[DataSecurity, Iso50001, GlobalWarming, Produc...",1,DTE,"[management facts, deutsche telekom cr report,...",101.78,Communication Services,Telecom Services,2021-03,2021,3,"[0.35756340622901917, 0.29088783264160156, 0.3...","[0.5, 0.0, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, ..."
2,Vonovia,sustainability_report,VonoviaSE Sustainability Report 2021,2021-03-31,NaN,"[Whistleblowing, DataSecurity, Vaccine, GHGEmi...",1,VNA,"[sustainable future, sustainability report dea...",20.35,Real Estate,Real Estate Services,2021-03,2021,3,"[0.4570336639881134, 0.45287153124809265, 0.26...","[0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.0, 0.5, 0.0, ..."
3,Merck,sustainability_report,MerckKGaA Sustainability Report 2021,2021-03-31,NaN,"[DataSecurity, DataMisuse, DrugResistance, Iso...",1,MRK,[management employees profile attractive emplo...,87.64,Healthcare,Drug Manufacturers—Specialty & Generic,2021-03,2021,3,"[0.36378589272499084, 0.6118267178535461, 0.48...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
4,MTU,sustainability_report,MTUAeroEngines Sustainability Report 2020,2020-03-31,NaN,"[WorkLifeBalance, Corruption, AirQuality, Data...",1,MTX,[sustainability goes far beyond climate action...,12.24,Industrials,Aerospace & Defense,2020-03,2020,3,"[0.46082836389541626, 0.46208637952804565, 0.4...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, ..."


### 1.3 Create different Dataframes (Sentences & full Document)

In [53]:
def create_sentence_df(data):

    # Select relevant columns
    data = data[['internal','sentence_tokens','sentiment_llm_categorial']]

    # Explode the tokens, so each sentence is a row
    data = data.set_index(['internal']).apply(pd.Series.explode).reset_index()

    # Rename the columns and change order
    data.rename(columns={'sentence_tokens': 'sentence', 'sentiment_llm_categorial': 'sentiment'}, inplace=True)
    data = data[['internal', 'sentence', 'sentiment']]

    # Convert types
    data['internal'] = data['internal'].astype(int)
    data['sentence'] = data['sentence'].astype(str)
    data['sentiment'] = data['sentiment'].astype(float)
    
    return data

# Create sentence data
sentence_df = create_sentence_df(df)

# Display header and shape
print(sentence_df.shape)
sentence_df.head()

(678529, 3)


,internal,sentence,sentiment
0,1,brands strategy sustainability agenda care bey...,0.5
1,1,successfully reduced carbon footprint absolute...,0.5
2,1,end consumer business returned levels reduced ...,0.0
3,1,decoupling human economic activity natural res...,0.0
4,1,inspired beiersdorf ambitious sustainability a...,0.5


In [54]:
# Function to create document data
def create_document_df(data):

    # Join tokens
    data['document'] = data['sentence_tokens'].apply(' '.join)  # Convert tokens to strings

    # Compute the mean of the computed sentiment and discretize it
    def discretize_sentiment(value):
        if value <= 0.33:
            return 0.0
        elif value <= 0.66:
            return 0.5
        else:
            return 1.0

    data['sentiment'] = data['sentiment_llm_continuous'].apply(np.mean).apply(discretize_sentiment)

    # Convert types
    data['internal'] = data['internal'].astype(int)
    data['sentence'] = data['document'].astype(str)
    data['sentiment'] = data['sentiment'].astype(float)

    # Return needed columns and discretized mean of the sentiment
    return data[['internal', 'document', 'sentiment']]

# Create sentence data
document_df = create_document_df(df)

# Display header and shape
print(document_df.shape)
document_df.head()

(11071, 3)


,internal,document,sentiment
0,1,brands strategy sustainability agenda care bey...,0.5
1,1,management facts deutsche telekom cr report th...,0.5
2,1,sustainable future sustainability report dear ...,0.5
3,1,management employees profile attractive employ...,0.5
4,1,sustainability goes far beyond climate action ...,0.5


The subsets for the training should have equally distributed classes. In addition, external and internal documents should be represented.  
One of these conditions needs to be more "loose", we decide class equality is more important.

In [79]:
def balance_sentiment_and_internal(df):
    # Get minimum number of observations across sentiment classes
    min_internal_count = df['internal'].value_counts().min()
    
    # Get the minimum number of observations between internal == 0 and internal == 1
    min_sentiment_count = min(df[df['sentiment'] == 0].shape[0], df[df['sentiment'] == 1].shape[0], min_internal_count)

    # Create "balanced" dataframe
    balanced_df = pd.concat([df[df['sentiment'] == i].sample(min_sentiment_count, random_state=1) for i in df['sentiment'].unique()])

    return balanced_df

# Sample the sentence dataframe
sub_sentence_df = balance_sentiment_and_internal(sentence_df)

# Display header and shape
print(sub_sentence_df.shape)
sub_sentence_df.head()

(103812, 3)


,internal,sentence,sentiment
482121,0,july incyte eli lilly announced fda would meet...,0.5
459567,0,still cautious highly contagious delta strain ...,0.5
110344,1,march first publicprivate peer employees tied ...,0.5
267939,0,content plans featuring presentation followed ...,0.5
315677,0,pubmed scopus google scholar however link nets...,0.5


In [80]:
# Sample the document dataframe
sub_document_df = balance_sentiment_and_internal(document_df)

# Display header and shape
print(sub_document_df.shape)
sub_document_df.head()

(186, 3)


,internal,document,sentiment
10179,0,dgapnews ag key word quarterly interim stateme...,0.5
2292,0,president fraunhofer institute ceramic technol...,0.5
10338,0,yet father granted biopic still waiting twenty...,0.5
5472,0,stepped gear dedicated taskforce held inaugura...,0.5
8299,0,business technology platform critical piece la...,0.5


In [81]:
# Inspect sampling results
print('Sentence subset:')
print(sub_sentence_df['internal'].value_counts())
print(sub_sentence_df['sentiment'].value_counts())
print('\n')
print('Document subset:')
print(sub_document_df['internal'].value_counts())
print(sub_document_df['sentiment'].value_counts())

Sentence subset:
0    72896
1    30916
Name: internal, dtype: int64
0.5    34604
0.0    34604
1.0    34604
Name: sentiment, dtype: int64


Document subset:
0    186
Name: internal, dtype: int64
0.5    62
1.0    62
0.0    62
Name: sentiment, dtype: int64


As a result, the models can be evaluated and trained with 2 approaches:  
- A dataframe containing the full document and a discretized mean sentiment of all included sentences
- A dataframe containing each sentence with the corresponding discretized sentiment  
- Two sampled subset dataframes for moel evaluation

"Discretized" corresponds to the labels 0.0 (negative), 0.5 (neutral) and 1.0 (positive).

## Model Evaluation

The evaluation for the model is based on the following conceptual approach:
1. Select multiple pretrained (Huggingface) models, based on previous stages
2. Train the selected models on a small subset of the full documents and the single sentences to keep the training time short
3. Compare the training outcomes of the different models on the two subsets and select the best model

In [84]:
# Function to compute the comparison metrics
def compute_metrics(p):
    pred, labels = p
    
    # Use the appropriate metrics, since we don't have discrete classes but a continous score 
    mse = mean_squared_error(y_true=labels, y_pred=pred)
    mae = mean_absolute_error(y_true=labels, y_pred=pred)
    r2 = r2_score(y_true=labels, y_pred=pred)

    return {"MSE": mse, "MAE": mae, "R2": r2}

In [86]:
# Load Tensorboard for training monitoring
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [135]:
# Kill potential Tensorboard process, so it don't block the port
!pkill -f "tensorboard"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [137]:
# Start Tensorboard to monitor training process
%tensorboard --logdir ./evaluation/ --port 6010

### Train Model 1: *distilbert-base-uncased*

As a first test, we use the lightweight "distilbert-base-uncased" model and fine-tune it on the full documents and the sentences, since the finetuned *"nlptown/bert-base-multilingual-uncased-sentiment"* demonstrated high alignment with gold standard in stage 2.  
Since BERT only accepts 512 input word tokens, the full documents are heavyily truncated.  

Model page: https://huggingface.co/distilbert-base-uncased

In [139]:
# Define pretrained tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1) # 1 label to get a continuous score between 0 and 1

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForSequenceClassification: ['distilbert.transformer.layer.4.sa_layer_norm.bias', 'distilbert.transformer.layer.4.sa_layer_norm.weight', 'distilbert.transformer.layer.0.output_layer_norm.bias', 'distilbert.transformer.layer.4.attention.out_lin.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.2.ffn.lin1.bias', 'distilbert.transformer.layer.3.attention.v_lin.bias', 'distilbert.transformer.layer.4.ffn.lin1.bias', 'vocab_tr

In [140]:
# Create the torch dataset to use dataset in PyTorch and override necessary methods
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

#### Train *distilbert-base-uncased* on sentence subset

In [141]:
# Split the data with a 70%, 15% and 15% ratio (train, valid, test)
X = list(sub_sentence_df["sentence"])
y = list(sub_sentence_df["sentiment"])
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3) # Split 70% train data
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5) # Split the other 30% in 50% each to get the correct ratio

# Tokenize the datasets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create the train, validation and test dataset as PyTorch datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)
test_dataset = Dataset(X_test_tokenized, y_test)

In [142]:
# Define training arguments
args = TrainingArguments(
    output_dir="./evaluation/distilbert_sentences",
    evaluation_strategy="steps",
    eval_steps=1000,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    gradient_accumulation_steps=4,
    seed=0,
    optim="adamw_torch", # Use newer PyTorch optimizer
    learning_rate=2e-5,
    logging_steps=10,
    fp16=True,
    report_to='tensorboard')

# Define Huggingface Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [143]:
# Delete GPU cache
torch.cuda.empty_cache()
# Train pre-trained model
trainer.train()

# Save the model
model.save_pretrained("./models/distilbert_sentences")

Step,Training Loss,Validation Loss


#### Train *distilbert-base-uncased* on document subset

In [97]:
# Split the data with a 70%, 15% and 15% ratio (train, valid, test)
X = list(sub_document_df["document"])
y = list(sub_document_df["sentiment"])
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3) # Split 70% train data
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5) # Split the other 30% in 50% each to get the correct ratio

# Tokenize the datasets
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create the train, validation and test dataset as PyTorch datasets
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)
test_dataset = Dataset(X_test_tokenized, y_test)

In [117]:
# Define training arguments
args = TrainingArguments(
    output_dir="./evaluation/distilbert_documents",
    evaluation_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=100,
    gradient_accumulation_steps=4,
    seed=0,
    optim="adamw_torch", # Use newer PyTorch optimizer
    learning_rate=2e-5,
    logging_steps=1,
    fp16=True,
    report_to='tensorboard')

# Define Huggingface Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [ ]:
# Delete GPU cache
torch.cuda.empty_cache()
# Train pre-trained model
trainer.train()

### Train Model 2: *roberta-base*

In [129]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset # Load default methods, since a few are overwritten for destillbert
import torch

In [131]:
# Load the tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def tokenize_and_format_sentence(examples):
    # Tokenize the text
    tokenized_inputs = tokenizer(examples['sentence'], truncation=True, padding='max_length')
    # Map the sentiment score to the label
    labels = examples['sentiment']
    # Return both the tokenized inputs and labels
    return {**tokenized_inputs, 'labels': labels}

# Train, validation and test split (70%, 15% and 15%)
train_dataset, temp_df = train_test_split(sentence_df, test_size=0.3, random_state=42)
val_dataset, test_dataset = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)

# Tokenizing the datasets
train_dataset = train_dataset.map(tokenize_and_format_sentence, batched=True)
val_dataset = val_dataset.map(tokenize_and_format_sentence, batched=True)

Map:   0%|          | 0/83050 [00:00<?, ? examples/s]

Map:   0%|          | 0/20762 [00:00<?, ? examples/s]

In [132]:
# Set the format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Load the base RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [138]:
# Prepare to train the model
args = TrainingArguments(
    output_dir="./evaluation/roberta_sentences",
    evaluation_strategy="steps",
    eval_steps=1000,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    gradient_accumulation_steps=4,
    seed=0,
    optim="adamw_torch", # Use newer PyTorch optimizer
    learning_rate=2e-5,
    logging_steps=10,
    fp16=True,
    report_to='tensorboard')

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./models/roberta_sentences")

Step,Training Loss,Validation Loss


### Compare the models

In [ ]:
# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Load trained model
model_path ="TBD"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=1)

# Define test trainer
test_trainer = Trainer(model)

# Make predictions
predictions = test_trainer.predict(test_dataset)

## Train, Dev, Test Split

In [ ]:
# Split the DataFrame into train, dev, and test sets (70%, 15% and 15%)
train_df, temp_df = train_test_split(sentence_df, test_size=0.3, random_state=42)
dev_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

## Tokenize Sentences

In [ ]:
# Define the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize the sentences and convert them to input features
def tokenize_sentences(sentences):
    input_ids = []
    attention_masks = []
    
    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=100,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Tokenize the sentences in the train, dev, and test sets
train_sentences = train_df['sentence'].tolist()
dev_sentences = dev_df['sentence'].tolist()
test_sentences = test_df['sentence'].tolist()
train_inputs, train_masks = tokenize_sentences(train_sentences)
dev_inputs, dev_masks = tokenize_sentences(dev_sentences)
test_inputs, test_masks = tokenize_sentences(test_sentences)

## Convert sentiment scores to tensors

In [ ]:
# Convert the sentiment scores to tensors
train_labels = torch.tensor(train_df['sentiment'].tolist())
dev_labels = torch.tensor(dev_df['sentiment'].tolist())
test_labels = torch.tensor(test_df['sentiment'].tolist())

## Create DataLoader and Load Data

In [ ]:
# Create a DataLoader for each set
batch_size = 64

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## Train base model

In [ ]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Set the device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f'\n Selected device to run: {device}')

In [ ]:
# Small evaluation function
def evaluate(model, dataloader):
    model.eval()

    total_loss = 0
    total_accuracy = 0

    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs, masks, labels = batch

        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks, labels=labels)

        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        # Calculate the accuracy for this batch
        predictions = torch.argmax(logits, dim=1).cpu().numpy()
        true_labels = labels.cpu().numpy()
        accuracy = accuracy_score(true_labels, predictions)
        total_accuracy += accuracy

    avg_loss = total_loss / len(dataloader)
    avg_accuracy = total_accuracy / len(dataloader)

    return avg_loss, avg_accuracy

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Set the optimizer and parameter
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 2

# Create the SummaryWriter
writer = SummaryWriter()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):        

        batch = tuple(t.to(device) for t in batch)
        inputs, masks, labels = batch

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        print(f'Epoch: {epoch+1}, Step: {step+1}, Loss: {loss.item()}')

        # Log the loss and learning rate to TensorBoard
        writer.add_scalar("Loss/train", loss, step)
        for param_group in optimizer.param_groups:
            writer.add_scalar("Learning rate", param_group['lr'], step)

        # Log histograms of all model parameters
        for name, param in model.named_parameters():
            if param.requires_grad:
                writer.add_histogram(name, param.data, step)
       

    avg_train_loss = total_loss / len(train_dataloader)
    writer.add_scalar("Average Loss/train", avg_train_loss, epoch)

    # Evaluate on the validation set and log metrics to TensorBoard
    avg_val_loss, avg_val_accuracy = evaluate(model, dev_dataloader)
    writer.add_scalar("Average Loss/validation", avg_val_loss, epoch)
    writer.add_scalar("Average Accuracy/validation", avg_val_accuracy, epoch)

# After training
trained_model = model
writer.close()

In [ ]:
print(trained_model)

## Evaluate base model on dev set

In [ ]:
# Evaluation on the dev set
trained_model.eval()
dev_predictions = []

with torch.no_grad():
    for batch in dev_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs, masks, labels = batch

        outputs = trained_model(inputs, attention_mask=masks)
        logits = outputs.logits

        predictions = torch.sigmoid(logits).squeeze().tolist()
        dev_predictions.extend(predictions)

    # Handle the remaining instances
    if len(dev_predictions) < len(dev_labels):
        remaining_instances = len(dev_labels) - len(dev_predictions)
        last_batch_inputs = dev_inputs[-remaining_instances:]
        last_batch_masks = dev_masks[-remaining_instances:]
        last_batch = (last_batch_inputs, last_batch_masks)

        last_batch = tuple(t.to(device) for t in last_batch)

        outputs = trained_model(*last_batch, attention_mask=last_batch[1])
        logits = outputs.logits

        predictions = torch.sigmoid(logits).squeeze().tolist()
        dev_predictions.extend(predictions)

# Calculate evaluation metrics
# Define the thresholds for discretization
thresholds = [1/3, 2/3]

# Discretize the predicted probabilities
discretized_predictions = np.digitize(dev_predictions, thresholds)

# Convert continuous labels to integers
dev_labels_int = np.digitize(dev_labels, thresholds)

# Create the confusion matrix-like representation
num_classes = len(thresholds) + 1  # Number of classes: below threshold, between thresholds, above threshold
cm = np.zeros((num_classes, num_classes))

for true_label, predicted_label in zip(dev_labels_int, discretized_predictions):
    cm[true_label, predicted_label] += 1

# Print the confusion matrix-like representation
print("Confusion Matrix:")
print(cm)

# Get the values from the confusion matrix
TP = cm[1, 1]
FP = cm[0, 1] + cm[2, 1]
FN = cm[1, 0] + cm[1, 2]
TN = cm[0, 0] + cm[0, 2] + cm[2, 0] + cm[2, 2]

# Compute accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Compute precision
precision = TP / (TP + FP)

# Compute recall
recall = TP / (TP + FN)

# Print the metrics
print(f"Accuracy:", accuracy)
print(f"Precision:", precision)
print(f"Recall:", recall)

## Parameter tuning

In [ ]:
# Define list of parameters
batch_sizes = [16]
learning_rates = [5e-5, 1e-5]
num_epochs_list = [1, 2]

# Set results dict
results = {
    'batch_size': [],
    'learning_rate': [],
    'num_epochs': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'confusion_matrix': []
}

# Iterate over parameter combinations
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        for num_epochs in num_epochs_list:
            # Train the model with the current hyperparameters
           
            # Load the pre-trained BERT model for sequence classification
            model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

            # Set the device (GPU if available, else CPU)
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            print(device)
            model = model.to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

            for epoch in range(num_epochs):
                model.train()
                for batch in train_dataloader:
                    batch = tuple(t.to(device) for t in batch)
                    inputs, masks, labels = batch
                    
                    optimizer.zero_grad()
                    # Forward pass
                    outputs = model(inputs, attention_mask=masks, labels=labels)
                    loss = outputs.loss
                    logits = outputs.logits
                    
                    # Backward pass and optimization
                    loss.backward()
                    optimizer.step()

            # Evaluation on dev set
            model.eval()
            dev_predictions = []

            with torch.no_grad():
                for batch in dev_dataloader:
                    batch = tuple(t.to(device) for t in batch)
                    inputs, masks, labels = batch

                    outputs = trained_model(inputs, attention_mask=masks)
                    logits = outputs.logits

                    predictions = torch.sigmoid(logits).squeeze().tolist()
                    dev_predictions.extend(predictions)

                # Handle the remaining instances
                if len(dev_predictions) < len(dev_labels):
                    remaining_instances = len(dev_labels) - len(dev_predictions)
                    last_batch_inputs = dev_inputs[-remaining_instances:]
                    last_batch_masks = dev_masks[-remaining_instances:]
                    last_batch = (last_batch_inputs, last_batch_masks)

                    last_batch = tuple(t.to(device) for t in last_batch)

                    outputs = trained_model(*last_batch, attention_mask=last_batch[1])
                    logits = outputs.logits

                    predictions = torch.sigmoid(logits).squeeze().tolist()
                    dev_predictions.extend(predictions)

            # Calculate evaluation metrics
            # Define the thresholds for discretization
            thresholds = [1/3, 2/3]

            # Discretize the predicted probabilities
            discretized_predictions = np.digitize(dev_predictions, thresholds)

            # Convert continuous labels to integers
            dev_labels_int = np.digitize(dev_labels, thresholds)

            # Create the confusion matrix-like representation
            num_classes = len(thresholds) + 1  # Number of classes: below threshold, between thresholds, above threshold
            cm = np.zeros((num_classes, num_classes))

            for true_label, predicted_label in zip(dev_labels_int, discretized_predictions):
                cm[true_label, predicted_label] += 1

            # Print the confusion matrix-like representation
            print("Confusion Matrix:")
            print(cm)

            # Get the values from the confusion matrix
            TP = cm[1, 1]
            FP = cm[0, 1] + cm[2, 1]
            FN = cm[1, 0] + cm[1, 2]
            TN = cm[0, 0] + cm[0, 2] + cm[2, 0] + cm[2, 2]

            # Compute accuracy
            accuracy = (TP + TN) / (TP + TN + FP + FN)

            # Compute precision
            precision = TP / (TP + FP)

            # Compute recall
            recall = TP / (TP + FN)

            # Store the results in the dictionary
            results['batch_size'].append(batch_size)
            results['learning_rate'].append(learning_rate)
            results['num_epochs'].append(num_epochs)
            results['accuracy'].append(accuracy)
            results['precision'].append(precision)
            results['recall'].append(recall)
            results['confusion_matrix'].append(cm)

# Create results df out of results dictionary
results_df = pd.DataFrame(results)

In [ ]:
# Display results dataframe
results_df

In [ ]:
# Create subplots for each metric
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
metrics = ['accuracy', 'precision', 'recall']
num_epochs = results_df['num_epochs'].unique().tolist()

for i, metric in enumerate(metrics):
    ax = axes[i // 2, i % 2]
    
    # Group the dataframe by batch_size and learning_rate
    grouped_df = results_df.groupby(['batch_size', 'learning_rate'])
    
    # Iterate over the unique combinations
    for (bs, lr), group in grouped_df:
        # Get the metric values for the current combination
        metric_values = group[metric].values
        
        # Plot the metric values as a line
        ax.plot(num_epochs, metric_values, marker='o', label=f"Batch sizes={bs}, LR={lr}")

    ax.set_xticks(num_epochs)
    ax.set_xticklabels(num_epochs)
    ax.set_xlabel("Number of epochs")
    ax.set_ylabel(metric.capitalize())
    ax.set_title(metric.capitalize())
    ax.legend()

plt.tight_layout()
plt.show()


## Train final model

In [ ]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Set the device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

# Set the optimizer and parameter
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3
batch_size = 16

In [ ]:
train_loss_values = []  # List to store training loss values

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0  # Variable to accumulate the loss for each epoch
    
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs, masks, labels = batch
        
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()  # Accumulate the loss for the current batch
        
    # Calculate the average training loss for the epoch
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    
    train_loss_values.append(avg_epoch_loss)  # Store the training loss value for the epoch
    
    # Print the training loss for the epoch
    print(f"Epoch {epoch+1} - Training Loss: {avg_epoch_loss:.4f}")

# After training
final_model = model


In [ ]:
print(final_model)

## Evaluate final model

In [ ]:
# Evaluation on the test set
final_model.eval()
test_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs, masks, labels = batch

        outputs = final_model(inputs, attention_mask=masks)
        logits = outputs.logits

        predictions = torch.sigmoid(logits).squeeze().tolist()
        test_predictions.extend(predictions)

    # Handle the remaining instances
    if len(test_predictions) < len(test_labels):
        remaining_instances = len(test_labels) - len(test_predictions)
        last_batch_inputs = test_inputs[-remaining_instances:]
        last_batch_masks = test_masks[-remaining_instances:]
        last_batch = (last_batch_inputs, last_batch_masks)

        last_batch = tuple(t.to(device) for t in last_batch)

        outputs = final_model(*last_batch, attention_mask=last_batch[1])
        logits = outputs.logits

        predictions = torch.sigmoid(logits).squeeze().tolist()
        test_predictions.extend(predictions)

# Calculate evaluation metrics
# Define the thresholds for discretization
thresholds = [1/3, 2/3]

# Discretize the predicted probabilities
discretized_predictions = np.digitize(test_predictions, thresholds)

# Convert continuous labels to integers
test_labels_int = np.digitize(test_labels, thresholds)

# Create the confusion matrix-like representation
num_classes = len(thresholds) + 1  # Number of classes: below threshold, between thresholds, above threshold
cm = np.zeros((num_classes, num_classes))

for true_label, predicted_label in zip(test_labels_int, discretized_predictions):
    cm[true_label, predicted_label] += 1

# Print the confusion matrix-like representation
print("Confusion Matrix:")
print(cm)

# Get the values from the confusion matrix
TP = cm[1, 1]
FP = cm[0, 1] + cm[2, 1]
FN = cm[1, 0] + cm[1, 2]
TN = cm[0, 0] + cm[0, 2] + cm[2, 0] + cm[2, 2]

# Compute accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)

# Compute precision
precision = TP / (TP + FP)

# Compute recall
recall = TP / (TP + FN)

# Print the metrics
print(f"Accuracy:", accuracy)
print(f"Precision:", precision)
print(f"Recall:", recall)

## Annotate sentiments with final prediction model

In [ ]:
# Define the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize the sentences and convert them to input features
def tokenize_sentences(sentences):
    input_ids = []
    attention_masks = []
    
    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=100,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

def make_predictions(tokenized_sentences):
  inputs, masks = tokenized_sentences

  # Prepare the data for model input
  inputs = inputs.to(device)
  masks = masks.to(device)

  # Evaluate the model on the dataframe
  final_model.eval()
  sentiment_predictions = []

  with torch.no_grad():
      for i in range(len(inputs)):
          input_ids = inputs[i].unsqueeze(0)
          attention_mask = masks[i].unsqueeze(0)
          
          outputs = final_model(input_ids, attention_mask=attention_mask)
          logits = outputs.logits
          
          predictions = torch.sigmoid(logits).squeeze().tolist()
          sentiment_predictions.append(predictions)
  return sentiment_predictions

subset_df['tokenized_sentences'] = subset_df['sentence_tokens'].apply(lambda x: tokenize_sentences(x))
subset_df['sentiments'] = subset_df['tokenized_sentences'].apply(lambda x: make_predictions(x))
subset_df.head()

## Compare internal vs. external

In [ ]:
# Compute the average of each list in the 'sentiments' column
subset_df['sentiments_avg'] = subset_df['sentiments'].apply(lambda x: np.mean(x))

# Create two separate dataframes for internal values 0 and 1
subset_internal_0 = subset_df[subset_df['internal'] == 0]
subset_internal_1 = subset_df[subset_df['internal'] == 1]

# Create boxplots for average sentiments grouped by internal values
plt.figure(figsize=(8, 6))
sns.boxplot(x='internal', y='sentiments_avg', data=subset_df)
plt.xlabel('Internal')
plt.ylabel('Average Sentiments')
plt.title('Boxplot of Average Sentiments by Internal')
plt.show()

# Create histograms for average sentiments grouped by internal values
plt.figure(figsize=(8, 6))
sns.histplot(data=subset_df, x='sentiments_avg', hue='internal', element='step', bins=10, alpha=0.5, legend=True)
plt.xlabel('Average Sentiments')
plt.ylabel('Frequency')
plt.title('Histogram of Average Sentiments by Internal')
plt.show()
